In [87]:
# /*  */ not supported yet
# case insensitiviy is not supported yet
# does not support cases where bne is before the label (in that case we need 2 passes)
# immediate for vsetivli is assumed an integer but immediate for other instr is asssumed in hex
# supports '//' comments, blank lines, spaces between operands, ',', instructions, ...
def parser(instruction_filename, binary_filename):
    label_dict = {}
    real_line_number = 0 # after deleting comments
    with open(binary_filename, 'w') as g:
        with open(instruction_filename) as f:
            lines = f.readlines()
            for i in range(len(lines)):
                this_line = lines[i]
                stripped = this_line.lstrip()
                if len(stripped) == 0: # skip blank lines
                    pass
                elif not (stripped[0] == '/' and stripped[1] == '/'): # skip comments
                    parsed_line = stripped.split('//')[0] # skip the comment at the end of the line
                    if len(parsed_line.split(':')) == 2: # This is a label
#                         print('real line number', real_line_number)
                        label = parsed_line.split(':')[0]
                        parsed_line = parsed_line.split(':')[1].lstrip()
#                         label_line = i
                        label_line = real_line_number
                        label_dict[label] = label_line # adding to the dictionary 
                        binary = assembler(parsed_line, label_dict)
                        g.write(binary + '\n')
                        real_line_number += 1
                        print(parsed_line)
                    else: # no label
#                         print('real line number', real_line_number)
                        binary = assembler(parsed_line, label_dict)
                        g.write(binary + '\n')
                        real_line_number += 1
                        print(parsed_line)
    print(label_dict)

In [88]:
def assembler(instruction, label_dict):
    instruction_name = instruction.split()[0]
    if len(instruction.split()) == 5: #(4 operands) vsetivli
        if instruction_name == 'vsetivli':
            rd = instruction.split()[1].rstrip(',').rstrip(' ') # remove the trailing ',' and then 'space(s)'
            imm_VLEN = int(instruction.split()[2].rstrip(',')) # remove the trailing ','
            if imm_VLEN < 0:
                print('Assebler does not support negative number for VLEN')
            num_element = instruction.split()[3].rstrip(',').rstrip(' ') # remove the trailing ',' and then 'space(s)'
            m_value = instruction.split()[4].split('\n')[0].rstrip(' ') # remove the trailing '\n' and then 'space(s)'
            if m_value == 'm2':
                m_binary = "{0:03b}".format(0)
            elif m_value == 'm4':
                m_binary = "{0:03b}".format(1)
            elif m_value == 'm8':
                m_binary = "{0:03b}".format(2)
            elif m_value == 'm16':
                m_binary = "{0:03b}".format(3)
            elif m_value == 'm32':
                m_binary = "{0:03b}".format(4)
            elif m_value == 'm64':
                m_binary = "{0:03b}".format(5)
            elif m_value == 'm128':
                m_binary = "{0:03b}".format(6)
            else:
                m_binary = "{0:03b}".format(7)
            return '11' + "{0:012b}".format(imm_VLEN) + m_binary + '111' + '00000' + '1010111'
        else:
            print('unsupported instruction')
    elif len(instruction.split()) == 4: #(3 operands) addi, add, vmacc.vx, bne
        if instruction_name == 'vmacc.vx':
            dest = instruction.split()[1].rstrip(',')
            vd = int(dest.lstrip('v'))
            source2 = instruction.split()[2].rstrip(',')
            vs2 = int(source2.lstrip('x'))
            return '1011000' + "{0:05b}".format(vs2) + '00000' + '000' + "{0:05b}".format(vd) + '1010111'
        elif instruction_name == 'add':
            dest = instruction.split()[1].rstrip(',')
            rd = int(dest.lstrip('x'))
            source1 = instruction.split()[2].rstrip(',')
            rs1 = int(source1.lstrip('x'))
            source2 = instruction.split()[3].split('\n')[0]
            rs2 = int(source2.lstrip('x'))
            return '0000000' + "{0:05b}".format(rs2) + "{0:05b}".format(rs1) + '000' + "{0:05b}".format(rd) + '0110011'
        elif instruction_name == 'addi':
            dest = instruction.split()[1].rstrip(',')
            rd = int(dest.lstrip('x'))
            source1 = instruction.split()[2].rstrip(',')
            rs1 = int(source1.lstrip('x'))
            imm_str = instruction.split()[3].split('\n')[0]
            imm = int(imm_str, base=16) # hex format
            if imm < 0:
                imm += 2**12 # 12 bit 2's complement
            return "{0:012b}".format(imm) + "{0:05b}".format(rs1) + '000' + "{0:05b}".format(rd) + '0010011'
        elif instruction_name == 'bne':
            source1 = instruction.split()[1].rstrip(',')
            rs1 = int(source1.lstrip('x'))
            source2 = instruction.split()[2].rstrip(',')
            rs2 = int(source2.lstrip('x'))
            label = instruction.split()[3].split('\n')[0].rstrip(' ')
            try:
                line_number = label_dict[label]
                branch_imm = "{0:012b}".format(line_number) # always positive
                return branch_imm[0] + branch_imm[2:8] + "{0:05b}".format(rs2) + "{0:05b}".format(rs1) + '001' + branch_imm[8:12] + branch_imm[1] + '1100011'
            except KeyError:
                print('Incorrect Label')
        else:
            print('unsupported instruction')
    
    elif len(instruction.split()) == 3: # (2 operands) lui, vle32.v   
        if instruction_name == 'lui':
            dest = instruction.split()[1].rstrip(',')
            rd = int(dest.lstrip('x'))
            imm_str = instruction.split()[2].split('\n')[0]
            imm = int(imm_str, base=16) # hex format
            if imm < 0:
                imm += 2**20 # 20 bit 2's complement
            return "{0:020b}".format(imm) + "{0:05b}".format(rd) + '0110111'
        elif instruction_name == 'vle32.v':
            dest = instruction.split()[1].rstrip(',')
            vd = int(dest.lstrip('v'))
            source1 = instruction.split()[2]
            source1_formatted = source1[source1.find('(')+1:source1.find(')')]
            rs1 = int(source1_formatted.lstrip('x'))
            return '000000000000' + "{0:05b}".format(rs1) + '000' + "{0:05b}".format(vd) + '0000111'
        else:
            print('unsupported instruction')
    elif len(instruction.split()) == 2: # (1 operand) streamout.v
        if instruction_name == 'streamout.v':
            source2 = instruction.split()[1].split('\n')[0]
            vs2 = int(source2.lstrip('v'))
            return '00000000000000000000' + "{0:05b}".format(vs2) + '1111111' 
        else:
            print('unsupported instruction')
    elif len(instruction.split()) == 1: # (0 operand) wfi
        return '00010000010100000000000001110011'
    else: 
        print("Error: instruction with more than 4 operands is not allowed")

In [97]:
# instruction_filename = './Assembly_code/gcn_xv_used.asm' #OS
instruction_filename = './Assembly_code/gcn_vx_used.asm' #WS
binary_filename = './Assembly_code/gcn_xv.bin'
parser(instruction_filename, binary_filename)

addi x5, x0, 0x000 
addi x6, x0, 0x001 
addi x30, x0, 0x017 
addi x29, x0, 0x000 
addi x28, x0, 0x800 
vsetivli zero, 1024, e32, m2 
vle32.v v1, (x0) 
addi x29, x0, 0x000 
lui x7, 0x00001 
vsetivli zero, 32, e32, m2 
vle32.v v0, (x7) 
add x7, x7, x28 
vsetivli zero, 3, e32, m2
vmacc.vx v1, x0, vs 
vmacc.vx v1, x1, vs

vmacc.vx v1, x2, vs

vmacc.vx v1, x3, vs

vmacc.vx v1, x4, vs

vmacc.vx v1, x5, vs

vmacc.vx v1, x6, vs

vmacc.vx v1, x7, vs

vmacc.vx v1, x8, vs

vmacc.vx v1, x9, vs

vmacc.vx v1, x10, vs

vmacc.vx v1, x11, vs

vmacc.vx v1, x12, vs

vmacc.vx v1, x13, vs

vmacc.vx v1, x14, vs

vmacc.vx v1, x15, vs

vmacc.vx v1, x16, vs

vmacc.vx v1, x17, vs

vmacc.vx v1, x18, vs

vmacc.vx v1, x19, vs

vmacc.vx v1, x20, vs

vmacc.vx v1, x21, vs

vmacc.vx v1, x22, vs

vmacc.vx v1, x23, vs

vmacc.vx v1, x24, vs

vmacc.vx v1, x25, vs

vmacc.vx v1, x26, vs

vmacc.vx v1, x27, vs

vmacc.vx v1, x28, vs

vmacc.vx v1, x29, vs

vmacc.vx v1, x30, vs

vmacc.vx v1, x31, vs

addi x29, x29, 0x001 
bne x2

In [98]:
num_col = 2
for i in range(num_col):
    !cp Assembly_code/gcn_xv.bin Assembly_code/gcn_xv_{str(i)}.bin
    !cp Assembly_code/gcn_xv_{str(i)}.bin /home/pouya/Documents/CGRA_binaries
# !rm Assembly_code/gcn_xv.bin
# !cp Assembly_code/*.bin /home/pouya/github/G-FPin_HW_old/G-FPin_HW/NetFPGA-PLUS/hw/lib/std/CGRA_v1_0_0/test/instructions/Cora/12
# !cp Assembly_code/*.bin /home/pouya/Documents/CGRA_binaries

In [78]:
instruction_filename = './Assembly_code/collective.asm'
binary_filename = './Assembly_code/collective_0.bin'
parser(instruction_filename, binary_filename)

addi x5, x0, 0x000 

addi x6, x0, 0x001 
addi x30, x0, 0x240 
addi x29, x0, 0x000 

vsetivli zero, 256, e32, m2 
vle32.v v0, (x0) 

addi x29, x0, 0x000 

vmacc.vx v0, x0, vs 

addi x29, x29, 0x001 

bne x29, x30, LOOP_TK 

streamout.v v0 

addi x5, x5, 0x001 

bne x5, x6, LOOP_TM 

wfi

{'LOOP_TM': 5, 'LOOP_TK': 7}


In [37]:
!cp Assembly_code/collective_0.bin /home/pouya/github/G-FPin_HW_old/G-FPin_HW/NetFPGA-PLUS/hw/lib/std/CGRA_v1_0_0/test/instructions/Cora/12